In [2]:
import numpy as np
import struct
import SimpleITK as sitk

# BDF file path
filename = "bdf.bdf"

# read BDF file
with open(filename, "rb") as f:
    data = f.read()

size = len(data)

image_size = []
spacing = []

# First 12 bytes define image sizes
for i in range(0,12,4):
    tmp = data[i:i+4]
    number = struct.unpack('<I', tmp)
    image_size.append(number[0])

# 12 to 24 bytes define pixel spacing
for i in range(12,24,4):
    tmp = data[i:i+4]
    number = struct.unpack('f', tmp)
    spacing.append(number[0])

print("array size ",size)
print("image_size", image_size)
print("spacing", spacing)

# The size of array should be 24 bytes + number of pixel*4 bytes* 3 dimension of vector
pixel_number = np.prod(image_size)
print(f"number of pixel = {pixel_number}")
assert pixel_number*4*3 + 24 == size

# Get image information
vector = []
vec_arr = []
count = 0

for i in range(24,size,4):
    tmp = data[i:i+4]
    number = struct.unpack('f', tmp)
    
    vector.append(number)
    count+=1
    
    if count ==3:
        vec_arr.append(vector)
        count = 0
        vector = []

# The number of pixel should equal length of vector
assert len(vec_arr) == pixel_number

# Create numpy image
vec_arr = np.array(vec_arr)
vec_arr = np.reshape(vec_arr, (*image_size[::-1], 3))

array size  402653208
image_size [512, 512, 128]
spacing [0.9699999690055847, 0.9699999690055847, 2.5]


In [3]:

# Create SITK image from numpy array
def image_from_array(array, origin, spacing):
    image = sitk.GetImageFromArray(array, isVector=True)
    image.SetSpacing(spacing)
    image.SetOrigin(origin)
    return image

In [5]:
image = image_from_array(vec_arr, (0,0,0),spacing  )
sitk.WriteImage(image, "image_test.nrrd")

transform = sitk.DisplacementFieldTransform(image)
sitk.WriteTransform(transform, 'transform_test.tfm')